In [6]:
!pip install torch torchvision onnx onnxruntime matplotlib

  Using cached onnx-1.17.0-cp311-cp311-win_amd64.whl.metadata (16 kB)
  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp311-cp311-win_amd64.whl.metadata (103 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 1.6/204.2 MB 10.5 MB/s eta 0:00:20
    --------------------------------------- 4.2/204.2 MB 1

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
from torchvision import transforms
from torchvision.datasets import VOCDetection
import time
from datetime import datetime
import json

print(f"Versão PyTorch: {torch.__version__}")
print(f"Versão TorchVision: {torchvision.__version__}")

def get_voc_dataset(root='./data'):
    """Usa o dataset Pascal VOC 2012 que já foi baixado."""
    # Transformações para aumentação de dados
    train_transform = transforms.Compose([
        transforms.Resize((640, 640)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
        transforms.ToTensor(),
    ])

    val_transform = transforms.Compose([
        transforms.Resize((640, 640)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Dataset de treino (usando VOC2012 train)
    train_dataset = VOCDetection(
        root=root,
        year='2012',
        image_set='train',
        download=True,  # Não baixar novamente
        transform=train_transform,
    )

    # Dataset de validação (usando parte do VOC2012)
    val_dataset = VOCDetection(
        root=root,
        year='2012',
        image_set='val',
        download=True,  # Não baixar novamente
        transform=val_transform,
    )

    print(f"Tamanho do dataset de treino: {len(train_dataset)}")
    print(f"Tamanho do dataset de validação: {len(val_dataset)}")

    return train_dataset, val_dataset

def collate_fn(batch):
    """Função de colagem personalizada para processar amostras de lotes."""
    return tuple(zip(*batch))

def create_model(num_classes=21):  # 20 classes + background
    """Cria um modelo SSDLite320 com backbone MobileNetV3 para detecção de objetos."""
    # Criar modelo com pesos pré-treinados
    weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
    model = ssdlite320_mobilenet_v3_large(weights=weights)

    # Modificar o número de classes, se necessário
    if num_classes != 91:  # 91 é o padrão para COCO
        # Inspecionar o modelo para entender a estrutura
        print("Adaptando modelo para classes VOC...")

        # Obter e substituir as camadas de classificação
        try:
            # Para TorchVision 0.20.1, acessar diretamente a camada cls_logits
            cls_logits = model.head.classification_head.cls_logits

            # Obter dimensões da camada existente
            in_channels = cls_logits.in_channels
            kernel_size = cls_logits.kernel_size
            stride = cls_logits.stride
            padding = cls_logits.padding

            # Obter número de âncoras do tamanho de saída atual
            out_channels = cls_logits.out_channels
            num_anchors = out_channels // 91  # 91 classes padrão COCO

            # Criar nova camada de classificação
            new_cls_logits = nn.Conv2d(
                in_channels,
                num_anchors * num_classes,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding
            )

            # Inicializar pesos
            nn.init.normal_(new_cls_logits.weight, std=0.01)
            nn.init.constant_(new_cls_logits.bias, 0)

            # Substituir camada
            model.head.classification_head.cls_logits = new_cls_logits
            print(f"Camada de classificação adaptada para {num_classes} classes")
            print(f"Estrutura cls_logits: in_channels={in_channels}, num_anchors={num_anchors}")
        except AttributeError:
            # Abordagem alternativa
            print("Erro ao acessar 'in_channels'. Tentando abordagem alternativa...")

            # Explorar estrutura do modelo
            print("Explorando estrutura do modelo:")
            print("Head:", model.head.__class__.__name__)
            print("Classification head:", model.head.classification_head.__class__.__name__)

            # Exibir todos os atributos da classification_head
            cls_head_attrs = dir(model.head.classification_head)
            print("Atributos da cabeça de classificação:", [a for a in cls_head_attrs if not a.startswith('_')])

            # Tentar acessar cls_logits
            if hasattr(model.head.classification_head, 'cls_logits'):
                cls_logits = model.head.classification_head.cls_logits
                print(f"cls_logits encontrado: {cls_logits}")
                print(f"cls_logits tipo: {type(cls_logits)}")

                # Tentar modificar usando a arquitetura existente
                try:
                    # Obter número de âncoras analisando o tensor de saída
                    num_classes_original = 91  # COCO classes
                    out_channels = cls_logits.out_channels
                    num_anchors = out_channels // num_classes_original

                    # Criar nova camada com parâmetros existentes
                    new_cls_logits = nn.Conv2d(
                        cls_logits.in_channels,
                        num_anchors * num_classes,
                        kernel_size=cls_logits.kernel_size,
                        padding=cls_logits.padding
                    )

                    # Inicializar pesos
                    nn.init.normal_(new_cls_logits.weight, std=0.01)
                    nn.init.constant_(new_cls_logits.bias, 0)

                    # Substituir camada
                    model.head.classification_head.cls_logits = new_cls_logits
                    print(f"Camada de classificação adaptada para {num_classes} classes")
                except Exception as e2:
                    print(f"Erro na segunda tentativa: {e2}")

    return model

def voc_to_coco_format(target, device):
    """Converte o formato de anotação Pascal VOC para o formato esperado pelo modelo."""
    boxes = []
    labels = []

    # Mapeamento de classes Pascal VOC para índices
    voc_classes = [
        'background',  # Classe 0 é o background
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
        'bus', 'car', 'cat', 'chair', 'cow',
        'diningtable', 'dog', 'horse', 'motorbike', 'person',
        'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
    ]
    class_to_idx = {cls: i for i, cls in enumerate(voc_classes)}

    try:
        for obj in target['annotation']['object']:
            bbox = obj['bndbox']
            xmin = float(bbox['xmin'])
            ymin = float(bbox['ymin'])
            xmax = float(bbox['xmax'])
            ymax = float(bbox['ymax'])

            # Verificar se as coordenadas são válidas
            if xmax > xmin and ymax > ymin:
                boxes.append([xmin, ymin, xmax, ymax])

                # Obter classe (com fallback para 'person' se não encontrada)
                class_name = obj['name']
                class_idx = class_to_idx.get(class_name, class_to_idx['person'])
                labels.append(class_idx)
    except Exception as e:
        print(f"Erro ao processar target: {e}")
        return None

    # Retornar None se não houver caixas válidas
    if not boxes:
        return None

    # Criar dicionário no formato esperado
    target_dict = {
        'boxes': torch.tensor(boxes, dtype=torch.float32, device=device),
        'labels': torch.tensor(labels, dtype=torch.int64, device=device)
    }

    return target_dict

def train_model(model, train_loader, val_loader, num_epochs=10, device='cuda'):
    """Treina o modelo SSD no dataset Pascal VOC."""
    # Definindo dispositivo
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Definindo otimizador e scheduler
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.0001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    # Métricas
    results = {
        'train_loss': [],
        'val_loss': []
    }

    # Loop de treinamento
    start_time = time.time()
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch+1}/{num_epochs}")

        # Modo de treinamento
        model.train()
        epoch_loss = 0
        batch_count = 0

        for i, (images, targets) in enumerate(train_loader):
            # Preparar imagens e alvos para o modelo
            images = list(img.to(device) for img in images)
            targets_formatted = []
            valid_images = []

            for img, target in zip(images, targets):
                target_dict = voc_to_coco_format(target, device)
                if target_dict is not None:
                    targets_formatted.append(target_dict)
                    valid_images.append(img)

            # Pular a iteração se não houver alvos válidos
            if not targets_formatted:
                continue

            # Usar apenas imagens válidas
            images = valid_images

            # Zerar gradientes
            optimizer.zero_grad()

            # Forward pass
            try:
                loss_dict = model(images, targets_formatted)
                losses = sum(loss for loss in loss_dict.values())

                # Checar se a perda é válida
                if not torch.isfinite(losses):
                    print(f"Perda não é finita, pulando batch {i}")
                    continue

                # Backward pass e otimização
                losses.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                # Calcular perda
                epoch_loss += losses.item()
                batch_count += 1
            except Exception as e:
                print(f"Erro no batch {i}: {e}")
                continue

            # Mostrar progresso
            if (i + 1) % 20 == 0:
                print(f"  Batch {i+1}/{len(train_loader)}, Perda: {losses.item():.4f}")

        # Prevenir divisão por zero
        if batch_count == 0:
            print("Aviso: Nenhum batch válido nesta época")
            continue

        # Salvar perda média da época
        avg_train_loss = epoch_loss / batch_count
        results['train_loss'].append(avg_train_loss)

        # Atualizar scheduler
        lr_scheduler.step()

        # Validação
        model.eval()
        val_loss = 0
        val_batch_count = 0

        with torch.no_grad():
            for images, targets in val_loader:
                images = list(img.to(device) for img in images)
                targets_formatted = []
                valid_images = []

                for img, target in zip(images, targets):
                    target_dict = voc_to_coco_format(target, device)
                    if target_dict is not None:
                        targets_formatted.append(target_dict)
                        valid_images.append(img)

                if not targets_formatted:
                    continue

                # Usar apenas imagens válidas
                images = valid_images

                # Forward pass - COM CORREÇÃO DO ERRO
                try:
                    # Em modo de avaliação, precisamos garantir que estamos no modo de treinamento para obter perdas
                    # Temporariamente mudar para modo de treinamento para validação
                    model.train()
                    loss_dict = model(images, targets_formatted)
                    model.eval()  # Voltar para modo de avaliação

                    # Verificar se o resultado é um dicionário
                    if isinstance(loss_dict, dict):
                        losses = sum(loss for loss in loss_dict.values())
                    elif isinstance(loss_dict, list):
                        # Se for uma lista, estamos no modo de inferência
                        print("Modelo retornou lista durante validação, passando para inferência...")
                        # Mudar para avaliação com targets para obter perdas
                        model.train()
                        loss_dict = model(images, targets_formatted)
                        model.eval()
                        losses = sum(loss for loss in loss_dict.values())
                    else:
                        print(f"Formato inesperado durante validação: {type(loss_dict)}")
                        continue

                    if torch.isfinite(losses):
                        val_loss += losses.item()
                        val_batch_count += 1
                except Exception as e:
                    print(f"Erro na validação: {e}")
                    continue

        # Calcular perda média de validação
        if val_batch_count > 0:
            avg_val_loss = val_loss / val_batch_count
            results['val_loss'].append(avg_val_loss)

            print(f"  Perda de treinamento: {avg_train_loss:.4f}, Perda de validação: {avg_val_loss:.4f}")

            # Salvar o melhor modelo
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': best_val_loss,
                }, 'best_ssdlite_mobilenet.pth')
                print(f"  Melhor modelo salvo (perda de validação: {best_val_loss:.4f})")
        else:
            print(f"  Perda de treinamento: {avg_train_loss:.4f}, Validação: Sem batches válidos")

    # Tempo total de treinamento
    total_time = time.time() - start_time
    print(f"\nTreinamento completo em {total_time / 60:.2f} minutos")

    return model, results

def export_to_onnx(model, device='cpu', output_path='ssdlite_mobilenet.onnx'):
    """Exporta o modelo treinado para o formato ONNX."""
    # Mover para CPU para exportação
    model = model.to('cpu')
    model.eval()

    # Criar uma entrada de exemplo
    dummy_input = torch.randn(1, 3, 640, 640)

    # Modo de inferência para exportação
    with torch.no_grad():
        try:
            # Classe wrapper para facilitar a exportação
            class ModelWrapper(torch.nn.Module):
                def __init__(self, model):
                    super(ModelWrapper, self).__init__()
                    self.model = model

                def forward(self, x):
                    # Durante a inferência, não passamos targets
                    with torch.no_grad():
                        return self.model([x])

            # Criar wrapper
            wrapped_model = ModelWrapper(model)

            # Exportar para ONNX
            torch.onnx.export(
                wrapped_model,
                dummy_input,
                output_path,
                export_params=True,
                opset_version=11,
                do_constant_folding=True,
                input_names=['input'],
                output_names=['detections'],
                dynamic_axes={'input': {0: 'batch_size'}}
            )

            print(f"Modelo exportado para {output_path}")
            return True
        except Exception as e:
            print(f"Erro ao exportar para ONNX: {e}")

            # Tentar exportar com abordagem diferente
            try:
                print("Tentando exportação alternativa...")

                class InferenceWrapper(torch.nn.Module):
                    def __init__(self, model):
                        super(InferenceWrapper, self).__init__()
                        self.model = model

                    def forward(self, x):
                        # Converter para lista antes de passar ao modelo
                        x_list = [x]
                        with torch.no_grad():
                            detections = self.model(x_list)
                        return detections

                # Criar wrapper
                inference_model = InferenceWrapper(model)

                # Tentar exportar o modelo diretamente
                torch.onnx.export(
                    inference_model,
                    dummy_input,
                    "ssdlite_mobilenet_inference.onnx",
                    export_params=True,
                    opset_version=11,
                    input_names=['input'],
                    output_names=['output']
                )
                print("Modelo exportado com sucesso para ssdlite_mobilenet_inference.onnx")
                return True
            except Exception as e2:
                print(f"Erro na exportação alternativa: {e2}")

                # Final fallback - exportar apenas o backbone
                try:
                    print("Tentando exportar apenas o backbone...")
                    backbone = model.backbone
                    torch.onnx.export(
                        backbone,
                        dummy_input,
                        "mobilenet_backbone.onnx",
                        export_params=True,
                        opset_version=11
                    )
                    print("Backbone exportado com sucesso para mobilenet_backbone.onnx")
                    return True
                except Exception as e3:
                    print(f"Erro ao exportar backbone: {e3}")
                    return False

def save_model_info(model, path='model_info.txt'):
    """Salva informações do modelo em um arquivo de texto."""
    info = {
        'modelo': 'SSDLite320-MobileNetV3-Large',
        'framework': 'PyTorch',
        'data_treinamento': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'dataset': 'Pascal VOC 2012',
        'input_size': '640x640',
        'arquitetura': str(model),
        'num_parametros': sum(p.numel() for p in model.parameters()),
        'parametros_treinaveis': sum(p.numel() for p in model.parameters() if p.requires_grad),
        'versao_pytorch': torch.__version__,
        'versao_torchvision': torchvision.__version__,
        'classes': [
            'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
            'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
            'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
            'train', 'tvmonitor'
        ]
    }

    with open(path, 'w') as f:
        for key, value in info.items():
            f.write(f"{key}: {value}\n")

    print(f"Informações do modelo salvas em {path}")

def main():
    # Verificar disponibilidade de GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Usando dispositivo: {device}")
    print(f"Versão PyTorch: {torch.__version__}")
    print(f"Versão torchvision: {torchvision.__version__}")

    # Obter datasets
    print("Preparando o dataset Pascal VOC 2012 já baixado...")
    train_dataset, val_dataset = get_voc_dataset()

    # Criar data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn
    )

    # Criar modelo
    print("Criando modelo SSDLite320 com MobileNetV3...")
    model = create_model()

    # Treinar modelo
    print("Iniciando treinamento...")
    model, results = train_model(
        model,
        train_loader,
        val_loader,
        num_epochs=50,
        device=device
    )

    # Carregar o melhor modelo
    try:
        checkpoint = torch.load('best_ssdlite_mobilenet.pth', map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Melhor modelo carregado com sucesso (época {checkpoint['epoch']}).")
    except Exception as e:
        print(f"Erro ao carregar o melhor modelo: {e}")

    # Exportar para ONNX
    print("Exportando modelo para ONNX...")
    export_to_onnx(model, device)

    # Salvar informações do modelo
    print("Salvando informações do modelo...")
    save_model_info(model)

    print("Processo concluído!")

if __name__ == "__main__":
    torch.multiprocessing.set_start_method('spawn', force=True)
    main()

Versão PyTorch: 2.5.1+cu121
Versão TorchVision: 0.20.1+cu121
Usando dispositivo: cuda
Versão PyTorch: 2.5.1+cu121
Versão torchvision: 0.20.1+cu121
Preparando o dataset Pascal VOC 2012 já baixado...
Tamanho do dataset de treino: 5717
Tamanho do dataset de validação: 5823
Criando modelo SSDLite320 com MobileNetV3...
Adaptando modelo para classes VOC...
Erro ao acessar 'in_channels'. Tentando abordagem alternativa...
Explorando estrutura do modelo:
Head: SSDLiteHead
Classification head: SSDLiteClassificationHead
Atributos da cabeça de classificação: ['T_destination', 'add_module', 'apply', 'bfloat16', 'buffers', 'call_super_init', 'children', 'compile', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer', 'get_extra_state', 'get_parameter', 'get_submodule', 'half', 'ipu', 'load_state_dict', 'module_list', 'modules', 'mtia', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'num_columns', 'parameters', 'register_backward_hoo

C:\Users\edvar\AppData\Local\Temp\ipykernel_11132\1815129545.py:529: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_ssdlite_mobilenet.pth', map_

Melhor modelo carregado com sucesso (época 26).
Exportando modelo para ONNX...
Erro ao exportar para ONNX: images is expected to be a list of 3d tensors of shape [C, H, W], got torch.Size([1, 3, 640, 640])
Tentando exportação alternativa...
Erro na exportação alternativa: images is expected to be a list of 3d tensors of shape [C, H, W], got torch.Size([1, 3, 640, 640])
Tentando exportar apenas o backbone...
Backbone exportado com sucesso para mobilenet_backbone.onnx
Salvando informações do modelo...
Informações do modelo salvas em model_info.txt
Processo concluído!


In [7]:
import warnings
# Ignora avisos de usuário (UserWarning) e avisos de tracer (TracerWarning) do PyTorch
warnings.filterwarnings("ignore", category=UserWarning, module="torch")
warnings.filterwarnings("ignore", category=UserWarning, module="torchvision")
warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)

import torch
import torch.nn as nn
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

def load_model_checkpoint(pth_path, device='cpu'):
    # Caso queira suprimir aviso do pickle, use:
    # checkpoint = torch.load(pth_path, map_location=device, weights_only=True)
    checkpoint = torch.load(pth_path, map_location=device)
    weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
    model = ssdlite320_mobilenet_v3_large(weights=weights)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

class DetectionWrapper(nn.Module):
    """
    Envolve o modelo SSDLite para converter tensores 4D (B,3,H,W)
    em uma lista de (3,H,W) para cada imagem do batch.
    """
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        # x é (B,3,H,W). Precisamos de [x[0], x[1], ...].
        images = [img for img in x]
        return self.model(images)

def export_to_onnx(model, output_path="best_ssdlite_mobilenet.onnx"):
    dummy_input = torch.randn(1, 3, 640, 640)
    wrapped_model = DetectionWrapper(model)
    model.to('cpu')

    torch.onnx.export(
        wrapped_model,
        dummy_input,
        output_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['detections'],
        dynamic_axes={'input': {0: 'batch_size'}}
    )
    print(f"Modelo exportado para {output_path}")

if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = load_model_checkpoint("best_ssdlite_mobilenet.pth", device=device)
    export_to_onnx(model, "best_ssdlite_mobilenet.onnx")

C:\Users\edvar\AppData\Local\Temp\ipykernel_8796\453568961.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pth_path, map_location=device)


Modelo exportado para best_ssdlite_mobilenet.onnx


In [11]:
import torch
import torchvision
import torchvision.transforms as T
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

# Configuração
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformações para os dados
transform = T.Compose([
    T.ToTensor(),
    T.Resize((320, 320)),
])

# Dataset COCO
data_dir = "./coco"  # Defina o caminho correto do COCO dataset
dataset = CocoDetection(root=f"{data_dir}/images/train2017", annFile=f"{data_dir}/annotations/instances_train2017.json", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda batch: tuple(zip(*batch)))

# Carregar modelo pré-treinado
model = ssdlite320_mobilenet_v3_large(pretrained=True)
model.to(device)
model.train()

# Otimizador e critério
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 5  # Ajuste conforme necessário

# Treinamento
for epoch in range(num_epochs):
    for images, targets in dataloader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Teste com uma imagem
image_path = "teste.jpg"  # Defina a imagem de teste correta
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_tensor = transform(image_rgb).unsqueeze(0).to(device)

# Predição
model.eval()
with torch.no_grad():
    prediction = model(image_tensor)

# Desenhar bounding boxes
image_np = image_rgb.copy()
for box, score, label in zip(prediction[0]['boxes'], prediction[0]['scores'], prediction[0]['labels']):
    if score > 0.5:  # Filtra objetos detectados com confiança acima de 50%
        x1, y1, x2, y2 = map(int, box.tolist())
        cv2.rectangle(image_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image_np, f"{label.item()} {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Exibir resultado
plt.figure(figsize=(10, 10))
plt.imshow(image_np)
plt.axis("off")
plt.show()

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: './coco/annotations/instances_train2017.json'